This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '____'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import requests
import json
import csv
#import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from bs4 import UnicodeDammit

In [ ]:
## I want to take a look in the metadata about Frankfurt Stock Exchange.
## https://www.quandl.com/data/FSE-Frankfurt-Stock-Exchange/usage/export
urlmeta = 'https://www.quandl.com/api/v3/databases/FSE/metadata?api_key=' + API_KEY
reqmeta = requests.get(urlmeta)

In [ ]:
## From metadata in reqmeta I want information about Carl Zeiss Meditec.
## First problem, reqmeta is zipped and need ZipFile to open.
## Second problem, response in reqmeta in bytes need BytesIO(reqmeta.content).
with ZipFile(BytesIO(reqmeta.content), 'r') as zipobj:
    listfiles = zipobj.namelist()
    print('List of files in metadata for Frankfurt Stock Exchange from quandl:')
    print(listfiles)
    
    if len(listfiles) == 1:
        print('\nInformation about Carl Zeiss Meditec in metadata:')
        ## I could have just used pandas reader here to create a df.
        #reader_df = pd.read_csv(zipobj.open(listfiles[0]))
        #print(reader_df)
        with zipobj.open(listfiles[0], 'r') as infile:
            ## I need to decode each line in infile using UnicodeDammit to find original code.
            ## I could have guessed something like utf8.
            #csvinfile = list(line.decode('utf8') for line in infile)
            csvinfile = list(line.decode(UnicodeDammit(line).original_encoding) for line in infile)
            reader_list = csv.reader(csvinfile)
            print(list(row for row in reader_list if 'editec' in str(row)))
            ## I could have created a dict.
            #reader_dict = csv.DictReader(csvinfile) 
            #print(list(reader_dict))  
    else:
        print('\nZero or more than 1 file in metadata!')

In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
urltest = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?' + \
      '&limit=1&api_key=' + API_KEY
reqtest = requests.get(urltest)

In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(reqtest.json())
## I could also have used content attribute.
#print(req.content)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [ ]:
## Task 1.
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + \
      "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
req = requests.get(url)

In [ ]:
## Task 2.
afx17 = req.json()
afx17a = json.loads(req.content)
afx_str = req.content.decode(UnicodeDammit(req.content).original_encoding)
afx17b = json.loads(afx_str)
afx17c = json.load(BytesIO(req.content))
print('Type of afx17: ', type(afx17))
print('Is afx17 same as afx17a? ', afx17 == afx17a)
print('Is afx17 same as afx17b? ', afx17 == afx17b)
print('Is afx17 same as afx17c? ', afx17 == afx17c)
print('\nFirst level keys of afx17: ', afx17.keys())
print('\nSecond level keys of afx17: ', afx17['dataset_data'].keys())

In [ ]:
## Preparing tuple of tuples for other tasks.
thenames = afx17['dataset_data']['column_names']
listoftuples = list(tuple(i) for i in afx17['dataset_data']['data'])
thetuple = tuple(sorted(listoftuples, key = lambda x: x[0], reverse=True))
print(thenames)
print(thetuple[0:5])

In [ ]:
## Task 3.
task3list = list(list(i) for i in thetuple if i[1] is not None)
maxopen = max(task3list, key = lambda x: float(x[1]))[1]
maxdate = max(task3list, key = lambda x: float(x[1]))[0]
minopen = min(task3list, key = lambda x: float(x[1]))[1]
mindate = min(task3list, key = lambda x: float(x[1]))[0]
print('The highest opening price was', maxopen, 'on', maxdate)
print('The lowest opening price was', minopen, 'on', mindate)

In [ ]:
## Task 4.
task4list = list(list(i) for i in thetuple if i[2] != None or i[3] != None)
for i in task4list:
    i.append(i[2] - i[3])
largchan = max(task4list, key = lambda x: float(x[11]))[11]
largdate = max(task4list, key = lambda x: float(x[11]))[0]
print('The largest change in any one day was', largchan, 'on', largdate)

In [ ]:
## Task 5.
task5list_today = list(list(i) for i in thetuple if i[4] is not None)
del task5list_today[-1]
task5list_yesterday = list(list(i) for i in thetuple if i[4] is not None)
del task5list_yesterday[0]
for i, j in zip(task5list_today, task5list_yesterday):
    i.append(abs(i[4] - j[4]))
largchan_2d = max(task5list_today, key = lambda x: float(x[11]))[11]
largdate_2d = max(task5list_today, key = lambda x: float(x[11]))[0]
print('The largest absolute change between any two days was', largchan_2d, 'on', largdate_2d)
print('Obs. 1: date shows the most recent day out of the two days being compared.')
print('Obs. 2: most but not all cases show day-right-after-day change.')

In [ ]:
## Task 6.
task6list = list(list(i) for i in thetuple if i[6] is not None)
tradevol = list(i[6] for i in task6list)
tradevolavg = sum(tradevol)/len(tradevol)
print('The average daily trading volume during 2017 was', tradevolavg)

In [ ]:
## Task 7 (using tradevol from task 6).
sortedtradevol = sorted(tradevol, reverse=False)

if isinstance(len(sortedtradevol)/2, int):
    tradevolmd = ( sortedtradevol[int(len(sortedtradevol)/2 - 1)] + \
                   sortedtradevol[int(len(sortedtradevol)/2    )] )/2
else:
    tradevolmd = sortedtradevol[int(len(sortedtradevol)/2 - 1/2)]

print('The median daily trading volume during 2017 was', tradevolmd)